This notebook will walk you through all of the steps necessary to take a catalog stored as a text file and generate GalSim images from it using the GalSim-CatSim interface.

The code for the CatSim-GalSim interface is stored in

    sims_GalSimInterface/python/lsst/sims/GalSimInterface/

<b>A cartoon of what you need to do</b>

In the file `galSimCatalogs.py` there are `InstanceCatalog` classes that are defined for different classes of object you would want to create images for.  The current defined classes are `GalSimStars` (for galactic point sources), `GalSimGalaxies` (for any object with a 2-D Sersic profile), and `GalSimAgn` (for extra-galactic point sources).

To create GalSim images, you need to instantiate one of these catalogs (or, more likely, a daughter-class of one these catalogs) and write it out using the usual `write_catalog()` method.  This will produce a text file with a list of all of the objects and a bunch of data about each, including which detector saw them.  As the catalog is writing itself to text, it will also be creating images of each object and adding it to the final image file for each detector.  After calling `write_catalog()`, you can write the stored images to FITS files by calling `write_images()`.

<b>Note:</b> if you call `write_images()` before calling `write_catalog()`, you will get empty FITS files.  `write_images()` just outputs the images already stored in the system.  Only `write_catalog()` iterates over and draws the objects in the catalog.

I know that the above makes it sound like you can only generate images with one class of object at a time.  What actually happens when you instantiate a `GalSimCatalog` class is that it creates a `GalSimInterpreter` object.  The `GalSimInterpreter` is the class which actually does the work of creating the images.  If you wanted to create a series of GalSim images with, for instance, both stars and galaxies in them, you can use the `GalSimCatalog` method `copyGalSimInterpreter` to pass the `GalSimInterpreter` from one catalog to another, i.e you would (in pseudo-code):

    create GalSimCatalogStars
    GalSimCatalogStars.write_catalog()
    
    create GalSimCatalogGalaxies
    GalSimCatalogGalaxies.copyGalSimInterpreter(GalSimCatalogStars)
    GalSimCatalogGalaxies.write_catalog()
    
    GalSimCatalogGalaxies.write_images()

<b>Interfacing with an arbitrary input catalog</b>

Obviously, the default `GalSimCatalog` classes are written to interface with the LSST database, bandpass, and SED libraries.  In order to customize the CatSim-GalSim interface, you will have to create a `CatalogDBObject` to interface with your catalog.

`CatalogDBObject` is the class used by the LSST Sims stack to query SQL databases.  An example of using this class to interface with non-LSST databases can be found in `sims_catUtils/examples/tutorials/reading_in_custom_data.ipynb`.

https://github.com/lsst/sims_catUtils/blob/master/examples/tutorials/reading_in_custom_data.ipynb

If you wish to just use the LSST catalogs hosted at the University of Washington, replace the customized `CatalogDBObject` classes below with the usual classe defined in

    sims_catUtils/python/lsst/sims/catUtils/baseCatalogModels/

The columns that your `CatalogDBObject` will need to provide for each object are:

<b>For point sources:</b>

* uniqueId -- some unique identifier of each object (an int)

* sedFilename -- the name of the file containing the object's SED

* magNorm -- a normalizing magnitude of the object's SED (due to convention, this is the magnitude of the object in a bandpass that is essentially a delta function at 500nm; to create an example of this bandpass, you can use the class `Bandpass` from `sims_photUtils` and the method `imsimBandpass()`)

* raJ2000, decJ2000 -- the mean RA and Dec (in the International Celestial Reference System) of the object.

* properMotionRa, properMotionDec -- the proper motion of the object.

* parallax -- pretty self-explanatory

* radialVelocity

* galacticAv -- the V-band extinction in magnitudes due to Milky Way Dust

<b>For galaxies:</b>

All of that (minus proper motion, parallax, and radial velocity), plus

* internalAv -- the V-band extinction in magnitudes due to internal dust (if your object is a galaxy; can be zero if it is a star)

* internalRv -- the reddening due to internal dust (again, if your object is a galaxy)

* redshift

* majorAxis -- in radians

* minorAxis -- in radians

* sindex -- the Sersic profile index (a float)

* positionAngle -- in radians

* halfLightRadius -- in radians



The cells below show how to use `fileDBObject` to convert a catalog text file into a database file that CatSim can understand.  For a more detailed discussion of just this process, see the example notebook in

    sims_catUtils/examples/tutorials/reading_in_custom_data.ipynb

We will be using the cartoon catalog stored in

    testCatalogs/cartoonPointSource.dat

which contains the following data on a collection of simulated extended objects

* id -- an int uniquely identifying each object in the catalog
* ra -- in degrees
* dec -- in degrees
* sedName
* magNorm
* rv -- radial velocity in km/s
* mura -- proper motion in RA in arcsec per year
* mudec -- proper motion in Dec in arcsec per year
* px -- parallax in arcsec

<b>Note:</b> sqlalchemy does not like square brackets or hyphens in column names. Do not use them.

<b>Note:</b> Whenever CatSim creates a `CatalogDBObject` or `InstanceCatalog` daughter class, that class gets stored in a registry of all of the created classes.  In the context of iPython notebooks, this means that if you run a cell that defines one of those classes more than once, you will get an exception.  If you find yourself needing to run the cell below (or any cell that defines a `CatalogDBObject` or `InstanceCatalog` daughter class) again, you will have to restart your kernel.  My apologies.  I am not terribly pleased with this behavior, either.

In [ ]:
import numpy
from lsst.sims.catalogs.generation.db import fileDBObject

class PointSourceCatalogDBObject(fileDBObject):
    idColKey = 'id' #the column that uniquely identifies each object in your catalog
    
    objectTypeId = 5 #something that CatSim uses to assign unique identifiers across multiple
                     #catalogs.  Just give it any int value.
        
    tableid = 'testPointSources' #the table that will be created to store your catalog in the database
    
    raColName = 'ra' #the column containing right ascension (necessary for doing spatially
                     #constrained searches on your catalog
        
    decColName = 'dec' #ditto for declination
    
    #below we transform the contents of the input catalog into the units and columns
    #expected by CatSim.  All angles should be in radians.
    columns = [('raJ2000','ra*PI()/180.0', numpy.float),
               ('decJ2000', 'dec*PI()/180.0', numpy.float),
               ('sedFilename', 'sedName', str, 100),
               ('magNorm', None),
               ('radialVelocity', 'rv', numpy.float),
               ('properMotionRa', 'mura*PI()/64800000.0'),
               ('properMotionDec', 'mudec*PI()/64800000.0'),
               ('parallax', 'px*PI()/64800000.0')]

Please note that, because CatSim internally handles all angles in radians, we had to convert ra, dec, px, mura, and mudec into radians and radians per year.

Now we will instantiate our `fileDBObject` daughter class and tell it to read in `testInputCatalogs/cartoon.cat`.

In [ ]:
starDB = PointSourceCatalogDBObject('testInputCatalogs/cartoonPointSource.dat')

Show the columns available in our new `fileDBObject` instantiation.

In [ ]:
starDB.show_mapped_columns()

<b>Definition of magNorm</b>

The column `magNorm` is a fairly specialized column in the world of CatSim.  Basically: CatSim reads in SEDs that have arbitrary normalization, and then renormalizing them by forcing them to have a set magnitude (`magNorm`) in what we call the 'imSim bandpass'.  The imsim Bandpass can be realized using (it is effectively a delta function at 500 nm)

In [ ]:
from lsst.sims.photUtils import Bandpass

controlBandpass = Bandpass()
controlBandpass.imsimBandpass()

The code below reads in an SED and calculates its magnitude in the imsim bandpass

In [ ]:
import os
from lsst.sims.photUtils import Sed

spectrum = Sed()
spectrum.readSED_flambda(os.path.join(os.getcwd(),'testSEDs','myCustomSED_1.dat'))
print spectrum.calcMag(controlBandpass)

This code renormalizes the spectrum to have a magnitude 15 in the imsim bandpass

In [ ]:
fnorm = spectrum.calcFluxNorm(15.0, controlBandpass)
spectrum.multiplyFluxNorm(fnorm)
print spectrum.calcMag(controlBandpass)

<b>Creating a `GalSimCatalog`</b>

In order to turn your catalog into images, you must read it into a `GalSimCatalog` daughter class.  Because all of the objects in `testInputCatalogs/cartoonPointSource.dat` are stars, we will use `GalSimStars` as the base class for our customized `GalSimCatalog`.

In order to customize the `GalSimCatalog` for a specific telescope, you will have to specify some member variables in your class declaration.

* `photParams` -- this is an instantiation of the `PhotometricParameters` class.  This is a class which stores data like platescale (arcseconds per pixel), readnoise, dark current, etc.  You can find it in the `sims_photUtils` package if you want to examine it in more detail.


* `sedDir` -- this is the directory where your SED files are stored.  SED files should be two columns: wavelength in nanometers and ergs/cm^2/s/nm.


* `bandpassDir` -- this is the directory where your bandpass files are stored.  Bandpass files should be two columns: wavelength in nanometers and throughput (between 0 and 1)


* `bandpassNames` -- a list of strings denoting the shorthand name of your bandpasses (i.e. ['u', 'g', 'r', 'i', 'z'] for SDSS)


* `bandpassRoot` -- the root of the name of your bandpass files.  The `GalSimCatalog` will expect your bandpass files to be named something like

        for bp in bandpassNames:
            bandpassDir + bandpassRoot + bp + '.dat'


* `componentList` -- this is a place holder for future functionality; just initialize it as an empty list


* `camera` -- this is the `afwCameraGeom` object that store the information about your camera.  Below, we will use a model of the LSST camera.


* `PSF` -- GalSim draws point sources by convolving them with a PSF.  There are a set of specific classes written for this purpose and defined in `sims_GalSimInterface/python/lsst/sims/GalSimInterface/galSimPSF.py`.  For now, we will use one of those pre-written PSF classes.  Below, we will show you how to write your own PSF class and use it to generate images.

<b>Note:</b> as another place holder for future functionality, the `GalSimCatalog` will expect the files `atmos.dat` and `darksky.dat` to be in your bandpass directory.  They are not currently used (they are there for when we implement a more rigorous sky brightness model in the LSST stack).  Just copy the provided files into your bandpass directory and you will be good to go.

You will also need to define the method `get_sedFilepath` so that it returns the file path to the SED file for each object in your catalog relative to the `sedDir` member variable defined in the class definition.

I recommend copying the `get_galacticAv` method below into any class that you find.  The getters for E(B-V) are a little buggy.  That should be fixed in the next release.

In [ ]:
import os
from lsst.sims.GalSimInterface import GalSimStars, SNRdocumentPSF
from lsst.sims.photUtils import PhotometricParameters
from lsst.obs.lsstSim import LsstSimMapper

class PointSourceGalSimCatalog(GalSimStars):

    photParams = PhotometricParameters()
    PSF = SNRdocumentPSF()
    
    sedDir = os.path.join(os.getcwd(),'testSEDs')
    bandpassDir = os.path.join(os.getcwd(),'testBandpasses')
    bandpassNames = ['y']
    bandpassRoot = 'myCustomBandpass_'
    componentList = []
    camera = LsstSimMapper().camera

    def get_sedFilepath(self):
        # this method would be more complicated if there were sub-directories
        # of sedDir that were not stored in the database
        return self.column_by_name('sedFilename')
    
    def get_galacticAv(self):
        ra = self.column_by_name('raJ2000')
        if len(ra)==0:
            return []

        ebv = self.column_by_name('EBV')
        return 3.1*ebv

In order to actually create a catalog, we need an instantiation of the `ObservationMetaData` class to tell the `GalSimCatalog` where the telescope is pointing.  `boundType` and `boundLength` will determine the field of view queried from the `fileDBObject`.  Because the `GalSimCatalog` is associating objects with detectors, you just need to make sure that the returned field of view is larger than the field of view of the camera.  Do not worry about making the field of view exactly the same size as the camera's field of view.

In [ ]:
import time
from lsst.sims.utils import ObservationMetaData
start = time.clock()
obs = ObservationMetaData(pointingRA=52.0, pointingDec=-26.0,
                          boundType='circle', boundLength=2.0,
                          mjd=52000.0, rotSkyPos=37.0)

cat = PointSourceGalSimCatalog(starDB, obs_metadata=obs)
cat.write_catalog('testOutputCatalogs/point_source_test_catalog.txt')
cat.write_images(nameRoot='testImages/pointSourceImage')
print time.clock()-start

The cell below will have created a bunch of fits files in the current working directory.  The naming convention of these files will be

    nameRoot_detector_filter.fits

Some of them will be empty.  The `GalSimCatalog` is very conservative about determining which detectors see distended objects (i.e. it associates objects with detectors it does not need to, just in case some of their flux bleeds over).  Having run this example, the image that you want to look at is `testImages/pointSourceImage_R_2_2_S_1_1_y.fits`.  That image definitely contains objects.

The catalog `testOutputCatalogs/point_source_test_catalog.txt` will list all of the objects included in your images, including which detectors they were seen by.  This can help you find a specific object in your images (modulo the warning about empty images above).

<b>Including a PSF</b>

The `GalSimCatalog` incorporates PSFs using daughter classes of the `PSFbase` class defined in `sims_GalSimInterface/python/lsst/sims/GalSimInterface/galSimPSF.py`.  `PSFbase` defines a method `applyPSF`.  This method uses the method `_getPSF` to get an instantiation of a function with which to convolve the image.

To define your own PSF class, simply define a daughter class of `PSFbase` that includes a method `_getPSF`.  You can then assign your PSF to the `GalSimCatalog` using the member variable `PSF`.  We demonstrate this below.

In [ ]:
import galsim
from lsst.sims.GalSimInterface import PSFbase

class myCrazyPSF(PSFbase):
    """
    This class defines Gaussian PSF whose width and orientation
    varies depending on its position in the focal plane
    """
    wavelength_dependent = False

    def _getPSF(self, xPupil=None, yPupil=None, **kwargs):
        """
        xPupil is the x coordinate in the pupil plane, in radians,
        at which the PSF is to be instantiated
        
        yPupil is the y coordinate in the pupil plane, in radians,
        at which the PSF is to be instantiated
        
        This method returns a GalSim gaussian class with which to
        convolve a point source
        """
        gaussian = galsim.Gaussian(sigma=1.0)
        xp = numpy.abs(xPupil)
        yp = numpy.abs(yPupil)
        if xp is None:
            xp is 0.0
        if yp is None:
            yp = 0.0
            
        if xp<yp:
            minor = xp
            major = yp
        else:
            minor = yp
            major = xp

        _psf = gaussian.shear(q=(minor+1.0)/(major+1.0), beta=(xPupil+yPupil)*galsim.radians)
        return _psf

Now we define a new `GalSimCatalog` that uses the PSF class we defined above.

In [ ]:
class CrazyPSFCatalog(PointSourceGalSimCatalog):
    PSF = myCrazyPSF()

Now we will write out images using our test PSF.

In [ ]:
start = time.clock()
cat = CrazyPSFCatalog(starDB, obs_metadata=obs)
cat.write_catalog('testOutputCatalogs/crazy_psf_point_source_catalog.txt')
cat.write_images(nameRoot='testImages/testCrazyPSF')
print time.clock()-start

If you do not want to write your own PSF class from scratch, `sims_GalSimInterface/python/lsst/sims/GalSimInterface/galSimPSF.py` defines a general double Gaussian PSF that you can access via

In [ ]:
from lsst.sims.GalSimInterface import DoubleGaussianPSF
help(DoubleGaussianPSF)

<b>Incorporating noise and background</b>

Noise and background are also incoporated using a special class of object, this time defined in `sims_GalSimInterface/python/lsst/GalSimInterface/galSimNoiseAndBackground.py`.  The classes defined there wrap the noise classes provided by GalSim into a form that the `GalSimCatalog` can use.  The example class `ExampleCCDNoise` is fairly general.

Noise and background classes are incorporated into the `GalSimCatalog` using the `noise_and_background` member variable.

In [ ]:
from  lsst.sims.GalSimInterface import ExampleCCDNoise
help(ExampleCCDNoise)

In [ ]:
class NoisyPointSourceCatalog(PointSourceGalSimCatalog):
    noise_and_background = ExampleCCDNoise(addNoise=True, addBackground=True)

Because we are no incorporating sky noise, we need to define the 5-sigma limiting magnitude `m5`.  This is added to the data stored in our `ObservationMetaData` instantiation.


In [ ]:
obs.setBandpassM5andSeeing(bandpassName=['x', 'y', 'z'], m5=[24.0]*3,
                           seeing=[0.7]*3)
cat = NoisyPointSourceCatalog(starDB, obs_metadata=obs)
cat.write_catalog('testOutputCatalogs/noisy_point_source_catalog.txt')
cat.write_images(nameRoot='testImages/testNoisyImage')

<b>Note:</b> `ObservationMetaData` also has a `Site` member that defaults to the LSST site.  You will probably want to customize that.  You can access the `Site` class using

In [ ]:
from lsst.sims.utils import Site
help(Site)

<b>Images with stars and galaxies</b>

If we want to incorporate galaxies into our test images, we must first read in a catalog of galaxies.

The catalog `testInputCatalogs/cartoonSersic.dat` contains the following columns

* id -- an int uniquely identifying each object in the catalog
* RA -- in degrees
* Dec -- in degrees
* PHOTO_Z -- redshift of the object
* Semi_major -- the semi major axis of the object in milli-arcseconds
* Semi_minor -- the semi minor axis of the object in milli-arcseconds
* PositionAngle -- the positiona angle of the object in degrees
* SersicIndex -- the index of the Sersic profile characterizing the object. <b>Note:</b> GalSim can only treat objects with Sersic indices between 0.3 and 6.2
* sedName -- the name of the file containing the SED of the object
* magNorm -- the normalizing magnitude of the object's spectrum

Below, we create a CatalogDBObject to read in that catalog and convert it to units that CatSim expects

In [ ]:
import numpy
from lsst.sims.catalogs.generation.db import fileDBObject

class SersicCatalogDBObject(fileDBObject):
    idColKey = 'id' #the column that uniquely identifies each object in your catalog
    
    objectTypeId = 4 #something that CatSim uses to assign unique identifiers across multiple
                     #catalogs.  Just give it any int value.
        
    tableid = 'testSersic' #the table that will be created to store your catalog in the database
    
    raColName = 'RA' #the column containing right ascension (necessary for doing spatially
                     #constrained searches on your catalog
        
    decColName = 'DEC' #ditto for declination
    
    #below we transform the contents of the input catalog into the units and columns
    #expected by CatSim.  All angles should be in radians.
    columns = [('raJ2000','RA*PI()/180.0', numpy.float),
               ('decJ2000', 'DEC*PI()/180.0', numpy.float),
               ('redshift', 'PHOTO_Z', numpy.float),
               ('majorAxis', 'Semi_Major_Axis*PI()/648000.0', numpy.float),
               ('minorAxis', 'Semi_Minor_Axis*PI()/648000.0', numpy.float),
               ('positionAngle', 'PositionAngle*PI()/180.0', numpy.float),
               ('sindex', 'SersicIndex', numpy.float),
               ('halfLightRadius', 'Semi_Major_Axis*PI()/648000.0', numpy.float),
               ('internalAv', '0.1', numpy.float),
               ('internalRv', '3.1', numpy.float),
               ('galacticRv', '3.1', numpy.float),
               ('sedFilename', 'sedName', str, 100),
               ('magNorm', None)]

Now we will create a GalSimCatalog to handle our extended objects

In [ ]:
from lsst.sims.GalSimInterface import GalSimGalaxies

class SersicGalSimCatalog(GalSimGalaxies):
    sedDir = os.path.join(os.getcwd(),'testSEDs')
    bandpassDir = os.path.join(os.getcwd(),'testBandpasses')
    bandpassNames = ['y']
    bandpassRoot = 'myCustomBandpass_'
    componentList = []

    #the PSF, nose_and_background, and camera will be copied over from NoisyPointSourceCatalog
    #when we copy the GalSimInterpreter, so we do not need to copy it here
    
    def get_sedFilepath(self):
        # this method would be more complicated if there were sub-directories
        # of sedDir that were not stored in the database
        return self.column_by_name('sedFilename')
    
    def get_galacticAv(self):
        ra = self.column_by_name('raJ2000')
        if len(ra)==0:
            return []

        ebv = self.column_by_name('EBV')
        return 3.1*ebv

In [ ]:
galDB = SersicCatalogDBObject('testInputCatalogs/cartoonSersic.dat')

In [ ]:
galDB.show_mapped_columns()

Below we create images that contain both point sources and extended sources.

In [ ]:
starCat = NoisyPointSourceCatalog(starDB, obs_metadata=obs)
starCat.write_catalog('testOutputCatalogs/just_stars.txt')

galCat = SersicGalSimCatalog(galDB, obs_metadata=obs)
galCat.copyGalSimInterpreter(starCat)
galCat.write_catalog('testOutputCatalogs/just_galaxies.txt')

galCat.write_images('testImages/compoundImage')

<b>Note:</b> If you have run this notebook, you have created a lot of FITS images in the sub-directory `testImages/`.  You may want to delete them, if you are done with this tutorial.